In [9]:
using DynamicPolynomials, SparseArrays, MatrixMarket

println("***Problem setting***")

data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data 
A = MatrixMarket.mmread(data*"/johnson16-2-4.mtx")

n=size(A,1)
for i in 1:n
    A[i,i]=1
end
@polyvar x[1:n]# variables

println("Number of variables: n=",n)

f=sum(A[i,j]*x[i]*x[j] for i=1:n for j=1:n)*1.0

            
g=[1.0-sum(x.^2)]
#type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)        

# unit sphere constraint
h=[1.0-sum(x)] #type of coefficients of each polynomial must be float

l=length(h)

println("Number of equality constraints: l=",l)

***Problem setting***
Number of variables: n=120
Number of inequality constraints: m=1
Number of equality constraints: l=1


In [18]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);
# data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
# include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=2
s=122

@time opt_val1=InterRelax.RelaxDense_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek");

**Interrupted relaxation based on Handelman's Positivstellensatz**
Relaxation order: k=2
Sparsity order: s=122


Maximal matrix size:121
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 7381            
  Cones                  : 0               
  Scalar variables       : 7505            
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   

In [19]:
1/opt_val1

15.002619330741902

In [21]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);


k=0
s=122

@time opt_val1,opt_sol=InterRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false)

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=0
Sparsity order: s=122


Maximal matrix size:121
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 7381            
  Cones                  : 0               
  Scalar variables       : 7383            
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   

(0.06666666664703225, Float64[])

In [22]:
1/opt_val1

15.000000004417744

In [32]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);


k=1

opt=InterRelax.TSSOS_Dense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,k);

**Semidefinite relaxation based on Putinar's Positivstellensatz**
Relaxation order: k=1
Maximal matrix size: 121
************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[120]
[1]
------------------------------------------------------
Assembling the SDP...
There are 7381 affine constraints.
SDP assembling time: 0.016831664 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 7381            
  Cones                  : 0               
  Scalar variables       : 123             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 

In [33]:
1/opt

-0.08982035925596626

In [23]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

using TSSOS

k=2

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 300
************************TSSOS************************


TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[23]
[1]
------------------------------------------------------
Assembling the SDP...
There are 17550 affine constraints.
SDP assembling time: 0.387775922 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 17550           
  Cones                  : 0               
  Scalar variables       : 301             
  Matrix variables       : 25              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1    

InterruptException: InterruptException:

In [31]:
1/opt

-0.6289652432943494